In [1]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain_openai.chat_models import ChatOpenAI

In [2]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [3]:
class Scientist(BaseModel):
    # The descriptions help the LLM determine what field is which
    name: str = Field(description="Name of a scientist")
    discoveries: list = Field(description="Python list of discoveries")
    

In [4]:
parser = PydanticOutputParser(pydantic_object=Scientist)

In [5]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "Name of a scientist", "title": "Name", "type": "string"}, "discoveries": {"description": "Python list of discoveries", "items": {}, "title": "Discoveries", "type": "array"}}, "required": ["name", "discoveries"]}
```


In [6]:
human_prompt = HumanMessagePromptTemplate.from_template("{request}\n{format_instructions}")

In [7]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [15]:
request = chat_prompt.format_prompt(request="Tell me about a famous scientist",
                                    format_instructions=parser.get_format_instructions()).to_messages()

In [16]:
model = ChatOpenAI()

In [17]:
result = model.invoke(request)

In [18]:
result.content

'{\n\t"name": "Isaac Newton",\n\t"discoveries": [\n\t\t"Law of Universal Gravitation",\n\t\t"Three Laws of Motion",\n\t"Optics: Reflecting Telescope"\n\t]\n}'

In [19]:
parser.parse(result.content)

Scientist(name='Isaac Newton', discoveries=['Law of Universal Gravitation', 'Three Laws of Motion', 'Optics: Reflecting Telescope'])